In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[8],6
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,5e-25
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[4]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`          
            layer        coolrg                layer    coolrg
pressure                                                      
0.000312        1 -7.515858e-06                    1  0.000059
0.000750        2 -5.710839e-06                    2  0.000182
0.001052        3 -1.357627e-06                    3  0.000018
0.001476        4  4.142950e-06                    4  0.000013
0.002070        5  1.092976e-05                    5  0.000009
0.002904        6  1.921399e-05                    6  0.000008
0.004074        7  2.922501e-05                    7  0.000009
0.005714        8  4.123467e-05                    8  0.000009
0.008015        9  5.554334e-05                    9  0.000012
0.011243       10  7.248909e-05                   10  0.000015
0.015771       11  9.245346e-05                   11  0.000019
0.022122       12  1.158478e-04                   12  0.000024
0.031031       13  1.430235e-04                   13  0.000028
0.043528       14  1.706832e-04                   14  0.000034
0.061057       15  1.946068e-04                   15  0.000042
0.085645       16  2.172514e-04                   16  0.000049
0.120136       17  2.417442e-04                   17  0.000055
0.168516       18  2.683940e-04                   18  0.000062
0.236378       19  2.972043e-04                   19  0.000069
0.331549       20  3.282347e-04                   20  0.000077
0.465100       21  3.580844e-04                   21  0.000083
0.652400       22  3.340073e-04                   22  0.000078
0.915100       23  2.491289e-04                   23  0.000058
1.283650       24  1.683928e-04                   24  0.000037
1.800600       25  1.089072e-04                   25  0.000022
2.525700       26  6.578057e-05                   26  0.000011
3.542800       27  3.531527e-05                   27  0.000003
4.969550       28  1.664418e-05                   28 -0.000002
6.970850       29  6.625056e-06                   29 -0.000004
9.778100       30  3.057423e-07                   30 -0.000006
13.715850      31 -4.466561e-06                   31 -0.000008
19.239350      32 -7.917435e-06                   32 -0.000009
26.987250      33 -9.095245e-06                   33 -0.000009
37.855300      34 -8.292995e-06                   34 -0.000009
53.100050      35 -7.043769e-06                   35 -0.000009
73.887500      36 -5.842187e-06                   36 -0.000010
97.662500      37 -5.112114e-06                   37 -0.000010
121.437500     38 -5.476312e-06                   38 -0.000010
145.212500     39 -8.941120e-06                   39 -0.000015
168.987500     40 -1.368351e-05                   40 -0.000020
192.762500     41 -2.014968e-05                   41 -0.000027
216.537500     42 -2.686264e-05                   42 -0.000033
240.312500     43 -3.459951e-05                   43 -0.000040
264.087500     44 -4.428045e-05                   44 -0.000049
287.862500     45 -5.513548e-05                   45 -0.000058
311.637500     46 -5.870089e-05                   46 -0.000063
335.412500     47 -4.329596e-05                   47 -0.000058
359.187500     48 -5.453565e-06                   48 -0.000038
382.962500     49  6.869390e-05                   49 -0.000004
406.737500     50  2.318023e-04                   50  0.000087
430.512500     51  5.206409e-04                   51  0.000279
454.287500     52  9.647159e-04                   52  0.000613
478.062500     53  1.579269e-03                   53  0.001130
501.837500     54  2.365945e-03                   54  0.001851
525.612500     55  3.314556e-03                   55  0.002763
549.387500     56  4.403591e-03                   56  0.003846
573.162500     57  5.608275e-03                   57  0.005054
596.937500     58  6.894007e-03                   58  0.006325
620.712500     59  8.223797e-03                   59  0.007591
644.487500     60  9.557536e-03                   60  0.008803
668.262500     61  1.085836e-02              

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -10.213940  0.000000e+00 -10.213940 -10.228395   
0.000624    2     -10.213940  1.494929e-09 -10.213940 -10.228395   
0.000876    3     -10.213940  2.148466e-09 -10.213940 -10.228395   
0.001229    4     -10.213940  3.228538e-09 -10.213940 -10.228395   
0.001723    5     -10.213940  5.014284e-09 -10.213940 -10.228395   
0.002417    6     -10.213940  7.960200e-09 -10.213940 -10.228395   
0.003391    7     -10.213940  1.281124e-08 -10.213940 -10.228395   
0.004757    8     -10.213940  2.078292e-08 -10.213940 -10.228395   
0.006672    9     -10.213940  3.385594e-08 -10.213940 -10.228395   
0.009359    10    -10.213940  5.525289e-08 -10.213940 -10.228395   
0.013128    11    -10.213940  9.020552e-08 -10.213940 -10.228395   
0.018415    12    -10.213940  1.471930e-07 -10.213940 -10.228395   
0.025830    13    -10.213940  2.399287e-07 -10.213940 -10.228395   
0.036232    14    -10.213940  3.904559e-07 -10.213940 -10.228395   
0.050823    15    -10.213940  6.303136e-07 -10.213940 -10.228395   
0.071291    16    -10.213940  1.001164e-06 -10.213939 -10.228395   
0.100000    17    -10.213940  1.566558e-06 -10.213938 -10.228395   
0.140271    18    -10.213940  2.427698e-06 -10.213937 -10.228395   
0.196760    19    -10.213939  3.738463e-06 -10.213935 -10.228395   
0.275997    20    -10.213938  5.731156e-06 -10.213933 -10.228395   
0.387100    21    -10.213937  8.754573e-06 -10.213928 -10.228395   
0.543100    22    -10.213935  1.330311e-05 -10.213922 -10.228394   
0.761700    23    -10.213932  1.930869e-05 -10.213913 -10.228394   
1.068500    24    -10.213930  2.594905e-05 -10.213904 -10.228394   
1.498800    25    -10.213928  3.282865e-05 -10.213895 -10.228394   
2.102400    26    -10.213927  3.989121e-05 -10.213888 -10.228395   
2.949000    27    -10.213928  4.706596e-05 -10.213881 -10.228396   
4.136600    28    -10.213930  5.429652e-05 -10.213876 -10.228398   
5.802500    29    -10.213935  6.187237e-05 -10.213873 -10.228401   
8.139200    30    -10.213941  7.047554e-05 -10.213871 -10.228405   
11.417000   31    -10.213951  8.070250e-05 -10.213871 -10.228411   
16.014700   32    -10.213966  9.306118e-05 -10.213873 -10.228420   
22.464000   33    -10.213988  1.083917e-04 -10.213879 -10.228433   
31.510500   34    -10.214018  1.293355e-04 -10.213889 -10.228452   
44.200100   35    -10.214062  1.609181e-04 -10.213902 -10.228480   
62.000000   36    -10.214126  2.094300e-04 -10.213916 -10.228523   
85.775000   37    -10.214213  2.800329e-04 -10.213933 -10.228584   
109.550000  38    -10.214302  3.552011e-04 -10.213947 -10.228650   
133.325000  39    -10.214395  4.328978e-04 -10.213963 -10.228720   
157.100000  40    -10.214528  5.407418e-04 -10.213988 -10.228821   
180.875000  41    -10.214709  6.825061e-04 -10.214026 -10.228959   
204.650000  42    -10.214947  8.641227e-04 -10.214083 -10.229143   
228.425000  43    -10.215236  1.077465e-03 -10.214159 -10.229369   
252.200000  44    -10.215579  1.322551e-03 -10.214256 -10.229639   
275.975000  45    -10.215986  1.605177e-03 -10.214381 -10.229962   
299.750000  46    -10.216470  1.934219e-03 -10.214536 -10.230347   
323.525000  47    -10.217045  2.343972e-03 -10.214702 -10.230806   
347.300000  48    -10.217726  2.902962e-03 -10.214823 -10.231352   
371.075000  49    -10.218524  3.684904e-03 -10.214839 -10.231998   
394.850000  50    -10.219687  5.041800e-03 -10.214645 -10.232952   
418.625000  51    -10.221460  7.467985e-03 -10.213992 -10.234424   
442.400000  52    -10.223989  1.146310e-02 -10.212526 -10.236538   
466.175000  53    -10.227394  1.758479e-02 -10.209809 -10.239406   
489.950000  54    -10.231748  2.638652e-02 -10.205362 -10.243098   
513.725000  55    -10.237068  3.837024e-02 -10.198698 -10.247635   
537.500000  56    -10.243309  5.394514e-02 -10.189363 -10.252957   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')